In [1]:
#importing all libraries

from bs4 import BeautifulSoup
import requests
import time
from random import randint
import urllib.request
from openpyxl import Workbook
import json

In [2]:
#url of sites that we want to scrap

sites = [
    f"https://engineering.careers360.com/colleges/list-of-engineering-colleges-in-india",
    f"https://engineering.careers360.com/colleges/list-of-engineering-colleges-in-india/?page=2&sort_by=4"
]

In [3]:
#creating a new excel workboot sheet

wb = Workbook()
ws = wb.active

#setting the work book title
ws.title = "Colleges"

#adding the title of the column in the sheet
ws.append(["Name" , "Location" , "State"  , "Type" , "Rating" ,  "Total Reviews" , "Reference Link" , "Courses Offered" ])


In [4]:
#a function that takes the url and web scrap all the courses and append the course name to the list degree

def get_course( url , degree) :

    try:

        res = requests.get(url)
        
        time.sleep(1)

        soup = BeautifulSoup( res.text , "html.parser" )

        course_list = soup.find_all( class_ = 'card_blk' )

        for course in course_list :
            course_details = course.find(class_ = "detail")

            degreeName = course_details.a.text
            duration   = course_details.span.text

            degree.append(degreeName)
            # print(degreeName , duration)

        try :
            isNextPageAvailable = soup.find_all(class_ = 'college_pagination')
            
            for i in range(len(isNextPageAvailable)) :
                get_course(isNextPageAvailable[i].find("a", class_ = 'pagination_list_last')["href"] , degree)

        except Exception as e:
            return degree

    except Exception as e:
        # print(f"Exception occured : from courses : {e} \n\n")
        # return
        return degree
    
    finally:
        return degree


In [5]:
# A function call that web scrap all the college detail from the url and append the details to the list
# A new function call is made to web scrap all the courses offered in the college.
# At last all the college details are been uploaded to the sheet in the excel

count = 0

def get_colleges(url):

    res = urllib.request.urlopen(url)
    time.sleep(3)
    
    res = urllib.request.urlopen(url)

    soup = BeautifulSoup( res.read() , "html.parser" )
    colleges_list = soup.find_all( class_ = 'card_block' )


    for college in colleges_list :
        
        try:

            try:
                collegeName = college.h3.text.split(",")[0]
            except AttributeError:
                collegeName = "N/A"


            try:
                college_details = college.find( class_ = 'content_block d-none d-md-block d-md-flex flex-row justify-content-between' ).text.split("  ")

                try:
                    location = college_details[0].split(",")[0]
                except Exception as e: 
                    location = "N/A"
                
                try:
                    state = college_details[0].split(",")[1]
                except Exception as e: 
                    state = "N/A"
                
                try:
                    college_type = college_details[1]
                except Exception as e: 
                    college_type = "N/A"
                
            except :
                college_type = "N/A"
                state = "N/A"
                location = "N/A"
            

            try:
                star = college.find( class_ = 'star_text').text.strip()
            except Exception as e:
                star = 0
            
            try :
                total_review = college.find(class_ = "review_text").a.text
            except Exception as e:
                total_review = 0
            
            try:
                college_ref_link = college.find(class_ = "links_list" ).a['href']
            except Exception as e:
                college_ref_link = "N/A"

            try:           
                courses_url = college.find(class_ = "links_list" ).find('a', text = 'Courses')['href']
                get_courses_offered_in_the_college = get_course( courses_url  , [] )
                
            
            except Exception as e:
                get_courses_offered_in_the_college = ["N/A"]


            # print(f'college name : {collegeName} ')
            # print(f'college location : {location} ')
            # print(f'college location : {state} ')
            # print(f'college type : {college_type} ')
            # print(f'star rating : {star} ' )
            # print(f'total review : {total_review}')
            # print('college reference link : ' + college_ref_link )
            # print('courses offered on the college : ')
            # print( get_courses_offered_in_the_college )
            
            # print("\n\n")
            # print(college)
            # print("\n\n")

            ws.append([
               collegeName ,
               location,
               state,
               college_type,
               star,
               total_review,
               college_ref_link,
               json.dumps(get_courses_offered_in_the_college)
            ])
        
        except Exception as e :
            print("exception received " , count)
            print(f"printing exception {e}")
            count += 1

    wb.save(r"E:\academic lab\IR\package\SampleCollegeData.xlsx")

In [6]:
# A loop statement that loops to the url and make a function call to scrap the content from the url

for i in sites:
    time.sleep(3)
    get_colleges(i)
    time.sleep(3)